In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import os

import haversine as hs

from datetime import datetime, timedelta

import dateutil.parser

In [2]:
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:
race_info = pd.read_csv("F1_23_Planned.csv")

race_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   year           24 non-null     int64  
 1   round          24 non-null     int64  
 2   race           24 non-null     object 
 3   date           24 non-null     object 
 4   month          24 non-null     int64  
 5   country        24 non-null     object 
 6   location       24 non-null     object 
 7   circuit        24 non-null     object 
 8   circuitRef     24 non-null     object 
 9   header         24 non-null     object 
 10  gap            24 non-null     int64  
 11  distance (km)  24 non-null     float64
 12  lat            24 non-null     float64
 13  lng            24 non-null     float64
 14  raceId         24 non-null     int64  
 15  circuitId      24 non-null     int64  
dtypes: float64(3), int64(6), object(7)
memory usage: 3.1+ KB


In [4]:
race_info.shape

(24, 16)

In [5]:
race_info

,year,round,race,date,month,country,location,circuit,circuitRef,header,gap,distance (km),lat,lng,raceId,circuitId
0,2023,1,Bahrain Grand Prix,2023-03-05,3,Bahrain,Sakhir,Bahrain International Circuit,bahrain,single,0,0.00,26.0325,50.51060,1098,3
1,2023,2,Saudi Arabian Grand Prix,2023-03-19,3,Saudi Arabia,Jeddah,Jeddah Corniche Circuit,jeddah,single,2,1258.43,21.6319,39.10440,1099,77
2,2023,3,Australian Grand Prix,2023-04-02,4,Australia,Melbourne,Albert Park Grand Prix Circuit,albert_park,single,2,12817.12,-37.8497,144.96800,1100,1
3,2023,4,Chinese Grand Prix,2023-04-16,4,China,Shanghai,Shanghai International Circuit,shanghai,single,2,8078.46,31.3389,121.22000,1012,17
4,2023,5,Azerbaijan Grand Prix,2023-04-30,4,Azerbaijan,Baku,Baku City Circuit,baku,double,2,6336.88,40.3725,49.85330,1101,73
5,2023,6,Miami Grand Prix,2023-05-07,5,USA,Miami,Miami International Autodrome,miami,double,1,11015.93,25.9581,-80.23890,1102,79
6,2023,7,Emilia Romagna Grand Prix,2023-05-21,5,Italy,Imola,Autodromo Enzo e Dino Ferrari,imola,triple,2,8172.92,44.3439,11.71670,1077,21
7,2023,8,Monaco Grand Prix,2023-05-28,5,Monaco,Monte-Carlo,Circuit de Monaco,monaco,triple,1,349.98,43.7347,7.42056,1104,6
8,2023,9,Spanish Grand Prix,2023-06-04,6,Spain,Montmeló,Circuit de Barcelona-Catalunya,catalunya,triple,1,485.64,41.5700,2.26111,1105,4
9,2023,10,Canadian Grand Prix,2023-06-18,6,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,single,2,5891.51,45.5000,-73.52280,1106,7


In [6]:
weather = pd.read_csv("Track Weather Info.csv")

weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   circuitRef                288 non-null    object 
 1   month                     288 non-null    int64  
 2   temp_mean (°C)            288 non-null    float64
 3   temp_max (°C)             288 non-null    float64
 4   temp_min (°C)             288 non-null    float64
 5   precipitation_mean (mm)   288 non-null    float64
 6   rain_mean (mm)            288 non-null    float64
 7   snow_mean (cm)            288 non-null    float64
 8   precipitation_total (mm)  288 non-null    float64
 9   rain_total (mm)           288 non-null    float64
 10  snow_total (cm)           288 non-null    float64
 11  moderate_days (>25.55°C)  288 non-null    float64
 12  high_days (>27.77°C)      288 non-null    float64
 13  precipitation_days        288 non-null    float64
 14  rain_days 

In [7]:
weather.shape

(288, 16)

In [8]:
weather.sample(10)

,circuitRef,month,temp_mean (°C),temp_max (°C),temp_min (°C),precipitation_mean (mm),rain_mean (mm),snow_mean (cm),precipitation_total (mm),rain_total (mm),snow_total (cm),moderate_days (>25.55°C),high_days (>27.77°C),precipitation_days,rain_days,snow_days
162,monza,7,23.20,27.99,17.84,8.25,8.25,0.00,80.19,80.19,0.00,5.97,1.70,9.40,9.40,0.00
28,bahrain,5,28.95,30.60,26.86,7.60,7.60,0.00,0.31,0.31,0.00,29.77,23.43,0.03,0.03,0.00
241,vegas,2,10.28,15.86,4.78,6.14,6.08,0.71,22.49,22.26,0.17,0.00,0.00,3.50,3.50,0.23
71,hungaroring,12,-0.04,2.78,-2.69,5.10,4.95,1.41,43.21,32.39,8.13,0.00,0.00,8.03,6.17,5.77
281,zandvoort,6,15.42,17.81,12.90,5.70,5.70,0.00,63.20,63.20,0.00,0.00,0.00,10.67,10.67,0.00
226,spa,11,5.39,7.70,3.30,5.62,5.43,1.43,79.83,71.43,6.17,0.00,0.00,13.80,12.70,4.30
272,yas_marina,9,33.37,40.24,27.12,0.00,0.00,0.00,0.17,0.17,0.00,30.00,30.00,0.00,0.00,0.00
69,hungaroring,10,10.69,14.97,6.82,6.69,6.65,1.56,47.12,46.90,0.16,0.00,0.00,6.73,6.73,0.10
55,catalunya,8,23.94,27.94,20.12,5.95,5.95,0.00,41.10,41.10,0.00,6.23,1.47,6.50,6.50,0.00
140,miami,9,27.05,29.45,25.24,8.18,8.18,0.00,180.33,180.33,0.00,28.80,6.03,21.67,21.67,0.00


In [9]:
f = '%Y-%m-%d'

race_info['date'] = pd.to_datetime(race_info['date'], format=f).dt.date

In [10]:
for index, row in race_info.iterrows():
        race_info.loc[index, 'month'] = race_info.loc[index, 'date'].month

In [11]:
race_info

,year,round,race,date,month,country,location,circuit,circuitRef,header,gap,distance (km),lat,lng,raceId,circuitId
0,2023,1,Bahrain Grand Prix,2023-03-05,3,Bahrain,Sakhir,Bahrain International Circuit,bahrain,single,0,0.00,26.0325,50.51060,1098,3
1,2023,2,Saudi Arabian Grand Prix,2023-03-19,3,Saudi Arabia,Jeddah,Jeddah Corniche Circuit,jeddah,single,2,1258.43,21.6319,39.10440,1099,77
2,2023,3,Australian Grand Prix,2023-04-02,4,Australia,Melbourne,Albert Park Grand Prix Circuit,albert_park,single,2,12817.12,-37.8497,144.96800,1100,1
3,2023,4,Chinese Grand Prix,2023-04-16,4,China,Shanghai,Shanghai International Circuit,shanghai,single,2,8078.46,31.3389,121.22000,1012,17
4,2023,5,Azerbaijan Grand Prix,2023-04-30,4,Azerbaijan,Baku,Baku City Circuit,baku,double,2,6336.88,40.3725,49.85330,1101,73
5,2023,6,Miami Grand Prix,2023-05-07,5,USA,Miami,Miami International Autodrome,miami,double,1,11015.93,25.9581,-80.23890,1102,79
6,2023,7,Emilia Romagna Grand Prix,2023-05-21,5,Italy,Imola,Autodromo Enzo e Dino Ferrari,imola,triple,2,8172.92,44.3439,11.71670,1077,21
7,2023,8,Monaco Grand Prix,2023-05-28,5,Monaco,Monte-Carlo,Circuit de Monaco,monaco,triple,1,349.98,43.7347,7.42056,1104,6
8,2023,9,Spanish Grand Prix,2023-06-04,6,Spain,Montmeló,Circuit de Barcelona-Catalunya,catalunya,triple,1,485.64,41.5700,2.26111,1105,4
9,2023,10,Canadian Grand Prix,2023-06-18,6,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,single,2,5891.51,45.5000,-73.52280,1106,7


In [12]:
race_info['month'] = race_info['month'].astype(int)

race_info

,year,round,race,date,month,country,location,circuit,circuitRef,header,gap,distance (km),lat,lng,raceId,circuitId
0,2023,1,Bahrain Grand Prix,2023-03-05,3,Bahrain,Sakhir,Bahrain International Circuit,bahrain,single,0,0.00,26.0325,50.51060,1098,3
1,2023,2,Saudi Arabian Grand Prix,2023-03-19,3,Saudi Arabia,Jeddah,Jeddah Corniche Circuit,jeddah,single,2,1258.43,21.6319,39.10440,1099,77
2,2023,3,Australian Grand Prix,2023-04-02,4,Australia,Melbourne,Albert Park Grand Prix Circuit,albert_park,single,2,12817.12,-37.8497,144.96800,1100,1
3,2023,4,Chinese Grand Prix,2023-04-16,4,China,Shanghai,Shanghai International Circuit,shanghai,single,2,8078.46,31.3389,121.22000,1012,17
4,2023,5,Azerbaijan Grand Prix,2023-04-30,4,Azerbaijan,Baku,Baku City Circuit,baku,double,2,6336.88,40.3725,49.85330,1101,73
5,2023,6,Miami Grand Prix,2023-05-07,5,USA,Miami,Miami International Autodrome,miami,double,1,11015.93,25.9581,-80.23890,1102,79
6,2023,7,Emilia Romagna Grand Prix,2023-05-21,5,Italy,Imola,Autodromo Enzo e Dino Ferrari,imola,triple,2,8172.92,44.3439,11.71670,1077,21
7,2023,8,Monaco Grand Prix,2023-05-28,5,Monaco,Monte-Carlo,Circuit de Monaco,monaco,triple,1,349.98,43.7347,7.42056,1104,6
8,2023,9,Spanish Grand Prix,2023-06-04,6,Spain,Montmeló,Circuit de Barcelona-Catalunya,catalunya,triple,1,485.64,41.5700,2.26111,1105,4
9,2023,10,Canadian Grand Prix,2023-06-18,6,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,single,2,5891.51,45.5000,-73.52280,1106,7


In [13]:
cols = race_info.columns.tolist()

print(cols)

['year', 'round', 'race', 'date', 'month', 'country', 'location', 'circuit', 'circuitRef', 'header', 'gap', 'distance (km)', 'lat', 'lng', 'raceId', 'circuitId']


In [14]:
cols = ['year', 'round', 'race', 'date', 'month', 'country', 'location', 'circuit', 'circuitRef', 'header', 'gap', 'distance (km)', 'lat', 'lng', 'raceId', 'circuitId']

race_info = race_info[cols]

race_info

,year,round,race,date,month,country,location,circuit,circuitRef,header,gap,distance (km),lat,lng,raceId,circuitId
0,2023,1,Bahrain Grand Prix,2023-03-05,3,Bahrain,Sakhir,Bahrain International Circuit,bahrain,single,0,0.00,26.0325,50.51060,1098,3
1,2023,2,Saudi Arabian Grand Prix,2023-03-19,3,Saudi Arabia,Jeddah,Jeddah Corniche Circuit,jeddah,single,2,1258.43,21.6319,39.10440,1099,77
2,2023,3,Australian Grand Prix,2023-04-02,4,Australia,Melbourne,Albert Park Grand Prix Circuit,albert_park,single,2,12817.12,-37.8497,144.96800,1100,1
3,2023,4,Chinese Grand Prix,2023-04-16,4,China,Shanghai,Shanghai International Circuit,shanghai,single,2,8078.46,31.3389,121.22000,1012,17
4,2023,5,Azerbaijan Grand Prix,2023-04-30,4,Azerbaijan,Baku,Baku City Circuit,baku,double,2,6336.88,40.3725,49.85330,1101,73
5,2023,6,Miami Grand Prix,2023-05-07,5,USA,Miami,Miami International Autodrome,miami,double,1,11015.93,25.9581,-80.23890,1102,79
6,2023,7,Emilia Romagna Grand Prix,2023-05-21,5,Italy,Imola,Autodromo Enzo e Dino Ferrari,imola,triple,2,8172.92,44.3439,11.71670,1077,21
7,2023,8,Monaco Grand Prix,2023-05-28,5,Monaco,Monte-Carlo,Circuit de Monaco,monaco,triple,1,349.98,43.7347,7.42056,1104,6
8,2023,9,Spanish Grand Prix,2023-06-04,6,Spain,Montmeló,Circuit de Barcelona-Catalunya,catalunya,triple,1,485.64,41.5700,2.26111,1105,4
9,2023,10,Canadian Grand Prix,2023-06-18,6,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,single,2,5891.51,45.5000,-73.52280,1106,7


In [27]:
race_info.to_csv("F1_23_Planned.csv", index=False)

In [16]:
race_info_weather = pd.merge(race_info, weather, on=['circuitRef', 'month'], how='left')

race_info_weather

,year,round,race,date,month,country,location,circuit,circuitRef,header,gap,distance (km),lat,lng,raceId,circuitId,temp_mean (°C),temp_max (°C),temp_min (°C),precipitation_mean (mm),rain_mean (mm),snow_mean (cm),precipitation_total (mm),rain_total (mm),snow_total (cm),moderate_days (>25.55°C),high_days (>27.77°C),precipitation_days,rain_days,snow_days
0,2023,1,Bahrain Grand Prix,2023-03-05,3,Bahrain,Sakhir,Bahrain International Circuit,bahrain,single,0,0.00,26.0325,50.51060,1098,3,20.41,22.20,18.39,4.69,4.69,0.00,9.19,9.19,0.00,0.23,0.00,1.70,1.70,0.00
1,2023,2,Saudi Arabian Grand Prix,2023-03-19,3,Saudi Arabia,Jeddah,Jeddah Corniche Circuit,jeddah,single,2,1258.43,21.6319,39.10440,1099,77,24.84,29.41,20.59,3.88,3.88,0.00,2.72,2.72,0.00,10.03,1.33,0.63,0.63,0.00
2,2023,3,Australian Grand Prix,2023-04-02,4,Australia,Melbourne,Albert Park Grand Prix Circuit,albert_park,single,2,12817.12,-37.8497,144.96800,1100,1,15.26,19.78,11.15,6.71,6.71,0.00,47.47,47.47,0.00,0.00,0.00,6.77,6.77,0.00
3,2023,4,Chinese Grand Prix,2023-04-16,4,China,Shanghai,Shanghai International Circuit,shanghai,single,2,8078.46,31.3389,121.22000,1012,17,15.29,20.19,10.91,10.48,10.48,0.00,106.85,106.85,0.00,0.00,0.00,10.03,10.03,0.00
4,2023,5,Azerbaijan Grand Prix,2023-04-30,4,Azerbaijan,Baku,Baku City Circuit,baku,double,2,6336.88,40.3725,49.85330,1101,73,12.13,14.49,9.88,4.26,4.26,0.88,25.67,25.34,0.24,0.00,0.00,5.50,5.40,0.27
5,2023,6,Miami Grand Prix,2023-05-07,5,USA,Miami,Miami International Autodrome,miami,double,1,11015.93,25.9581,-80.23890,1102,79,25.52,28.25,23.17,7.88,7.88,0.00,103.53,103.53,0.00,15.73,0.40,12.70,12.70,0.00
6,2023,7,Emilia Romagna Grand Prix,2023-05-21,5,Italy,Imola,Autodromo Enzo e Dino Ferrari,imola,triple,2,8172.92,44.3439,11.71670,1077,21,17.98,23.00,12.73,6.80,6.80,0.00,63.83,63.83,0.00,0.20,0.00,9.03,9.03,0.00
7,2023,8,Monaco Grand Prix,2023-05-28,5,Monaco,Monte-Carlo,Circuit de Monaco,monaco,triple,1,349.98,43.7347,7.42056,1104,6,17.35,19.96,14.14,5.90,5.89,0.35,63.02,62.98,0.02,0.00,0.00,10.17,10.17,0.07
8,2023,9,Spanish Grand Prix,2023-06-04,6,Spain,Montmeló,Circuit de Barcelona-Catalunya,catalunya,triple,1,485.64,41.5700,2.26111,1105,4,21.28,25.41,16.92,5.70,5.70,0.00,30.34,30.34,0.00,1.80,0.10,4.97,4.97,0.00
9,2023,10,Canadian Grand Prix,2023-06-18,6,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,single,2,5891.51,45.5000,-73.52280,1106,7,19.09,23.56,14.63,8.90,8.90,0.00,113.48,113.48,0.00,0.93,0.10,12.53,12.53,0.00


In [17]:
cols = weather.columns.tolist()

print(cols)

['circuitRef', 'month', 'temp_mean (°C)', 'temp_max (°C)', 'temp_min (°C)', 'precipitation_mean (mm)', 'rain_mean (mm)', 'snow_mean (cm)', 'precipitation_total (mm)', 'rain_total (mm)', 'snow_total (cm)', 'moderate_days (>25.55°C)', 'high_days (>27.77°C)', 'precipitation_days', 'rain_days', 'snow_days']


In [18]:
cols = ['temp_mean (°C)', 'temp_max (°C)', 'temp_min (°C)', 'precipitation_mean (mm)', 'rain_mean (mm)', 'snow_mean (cm)', 'precipitation_total (mm)', 'rain_total (mm)', 'snow_total (cm)', 'moderate_days (>25.55°C)', 'high_days (>27.77°C)', 'precipitation_days', 'rain_days', 'snow_days']

race_info_weather[cols].describe()

,temp_mean (°C),temp_max (°C),temp_min (°C),precipitation_mean (mm),rain_mean (mm),snow_mean (cm),precipitation_total (mm),rain_total (mm),snow_total (cm),moderate_days (>25.55°C),high_days (>27.77°C),precipitation_days,rain_days,snow_days
count,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000
mean,19.717083,23.717083,15.797917,7.350000,7.346667,0.076667,78.968750,78.949583,0.013333,4.956250,1.680000,9.307083,9.302917,0.018333
std,4.522862,4.501062,4.791727,3.132179,3.132083,0.221451,61.022427,61.039829,0.049927,9.125569,5.587159,6.092533,6.095284,0.058878
min,12.130000,14.490000,7.840000,1.800000,1.800000,0.000000,0.300000,0.300000,0.000000,0.000000,0.000000,0.070000,0.070000,0.000000
25%,16.927500,20.310000,12.665000,5.575000,5.575000,0.000000,29.172500,29.090000,0.000000,0.000000,0.000000,5.367500,5.292500,0.000000
50%,18.665000,23.015000,15.050000,6.765000,6.765000,0.000000,66.830000,66.830000,0.000000,0.215000,0.015000,9.630000,9.630000,0.000000
75%,22.105000,26.657500,18.835000,8.307500,8.307500,0.000000,108.507500,108.507500,0.000000,4.747500,0.525000,12.732500,12.732500,0.000000
max,29.610000,34.240000,25.780000,16.080000,16.080000,0.880000,223.340000,223.340000,0.240000,31.000000,27.230000,23.300000,23.300000,0.270000


In [19]:
for col in cols:
    
    q1 = round(race_info_weather[col].quantile(.25), 2)
    q3 = round(race_info_weather[col].quantile(.75), 2)
    iqr = round(q3-q1, 2)
    high = round(race_info_weather[col].max(), 2)
    
    print(col)
    print(q3, q1, iqr, round((q3 + (1.5*iqr)), 2), round((q1 - (1.5*iqr)), 2), high)
    
    for index, row in race_info_weather.iterrows():
        if row[col] > (q3 + (1.5*iqr)):
            print("High outlier :", row['circuitRef'], row[col])
                       
        if row[col] < (q1 - (1.5*iqr)):
            print("Low outlier :", row['circuitRef'], row[col])
    
    print()

temp_mean (°C)
22.1 16.93 5.17 29.86 9.18 29.61

temp_max (°C)
26.66 20.31 6.35 36.18 10.78 34.24

temp_min (°C)
18.84 12.66 6.18 28.11 3.39 25.78

precipitation_mean (mm)
8.31 5.58 2.73 12.4 1.49 16.08
High outlier : monza 12.42
High outlier : suzuka 16.08

rain_mean (mm)
8.31 5.58 2.73 12.4 1.49 16.08
High outlier : monza 12.42
High outlier : suzuka 16.08

snow_mean (cm)
0.0 0.0 0.0 0.0 0.0 0.88
High outlier : baku 0.88
High outlier : monaco 0.35
High outlier : vegas 0.61

precipitation_total (mm)
108.51 29.17 79.34 227.52 -89.84 223.34

rain_total (mm)
108.51 29.09 79.42 227.64 -90.04 223.34

snow_total (cm)
0.0 0.0 0.0 0.0 0.0 0.24
High outlier : baku 0.24
High outlier : monaco 0.02
High outlier : vegas 0.06

moderate_days (>25.55°C)
4.75 0.0 4.75 11.88 -7.12 31.0
High outlier : miami 15.73
High outlier : marina_bay 29.83
High outlier : losail 31.0
High outlier : yas_marina 15.6

high_days (>27.77°C)
0.52 0.0 0.52 1.3 -0.78 27.23
High outlier : jeddah 1.33
High outlier : marina_bay

In [20]:
for col in cols:
    
    q1 = round(race_info_weather[col].quantile(.25), 2)
    q3 = round(race_info_weather[col].quantile(.75), 2)
    iqr = round(q3-q1, 2)
    
    print(col)
    print(q3, q1, iqr)
    
    for index, row in weather.iterrows():
        if row['month'] < 3:
            continue
        
        if row[col] > (q3 + (1.5*iqr)):
            print("High outlier :", row['circuitRef'], row['month'],row[col])
                       
        if row[col] < (q1 - (1.5*iqr)):
            print("Low outlier :", row['circuitRef'], row['month'], row[col])
    
    print()
        

temp_mean (°C)
22.1 16.93 5.17
High outlier : bahrain 6 31.98
High outlier : bahrain 7 33.37
High outlier : bahrain 8 33.68
High outlier : bahrain 9 32.17
Low outlier : baku 3 7.98
Low outlier : baku 12 8.13
Low outlier : catalunya 12 8.63
Low outlier : hungaroring 3 4.9
Low outlier : hungaroring 11 5.07
Low outlier : hungaroring 12 -0.04
Low outlier : imola 12 5.65
High outlier : jeddah 6 30.67
High outlier : jeddah 7 31.86
High outlier : jeddah 8 32.1
High outlier : jeddah 9 31.07
High outlier : losail 5 31.23
High outlier : losail 6 33.56
High outlier : losail 7 34.81
High outlier : losail 8 34.78
High outlier : losail 9 32.8
Low outlier : monza 3 7.99
Low outlier : monza 11 7.66
Low outlier : monza 12 3.07
Low outlier : red_bull_ring 3 3.02
Low outlier : red_bull_ring 4 8.04
Low outlier : red_bull_ring 10 9.04
Low outlier : red_bull_ring 11 3.9
Low outlier : red_bull_ring 12 -1.02
Low outlier : shanghai 12 6.75
Low outlier : silverstone 3 5.88
Low outlier : silverstone 4 8.18
Low o

High outlier : monaco 3 4.09
High outlier : monaco 4 1.16
High outlier : monaco 5 0.02
High outlier : monaco 10 0.21
High outlier : monaco 11 1.47
High outlier : monaco 12 4.57
High outlier : monza 3 2.11
High outlier : monza 4 0.07
High outlier : monza 10 0.04
High outlier : monza 11 1.3
High outlier : monza 12 6.52
High outlier : red_bull_ring 3 18.51
High outlier : red_bull_ring 4 8.44
High outlier : red_bull_ring 5 0.45
High outlier : red_bull_ring 9 0.23
High outlier : red_bull_ring 10 3.64
High outlier : red_bull_ring 11 15.09
High outlier : red_bull_ring 12 22.3
High outlier : shanghai 3 0.38
High outlier : shanghai 12 0.7
High outlier : silverstone 3 2.7
High outlier : silverstone 4 0.39
High outlier : silverstone 10 0.04
High outlier : silverstone 11 0.62
High outlier : silverstone 12 2.51
High outlier : spa 3 11.3
High outlier : spa 4 3.41
High outlier : spa 5 0.26
High outlier : spa 10 0.19
High outlier : spa 11 6.17
High outlier : spa 12 18.73
High outlier : suzuka 3 0.11
H

In [21]:
for col in cols:
    
    q1 = round(race_info_weather[col].quantile(.25), 2)
    q3 = round(race_info_weather[col].quantile(.75), 2)
    iqr = round(q3-q1, 2)
    
    print(col)
    print(q3, q1, iqr)
    
    for index, row in weather.iterrows():
        if row['month'] < 3:
            continue
        
        if row[col] <= (q3 + (1.5*iqr)):
            if row[col] >= (q1 - (1.5*iqr)):
                print("Optimal :", row['circuitRef'], row['month'],row[col])
    
    print()
        

temp_mean (°C)
22.1 16.93 5.17
Optimal : albert_park 3 18.27
Optimal : albert_park 4 15.26
Optimal : albert_park 5 12.48
Optimal : albert_park 6 10.26
Optimal : albert_park 7 9.64
Optimal : albert_park 8 10.36
Optimal : albert_park 9 12.18
Optimal : albert_park 10 14.18
Optimal : albert_park 11 16.35
Optimal : albert_park 12 18.22
Optimal : americas 3 16.37
Optimal : americas 4 20.3
Optimal : americas 5 24.27
Optimal : americas 6 27.62
Optimal : americas 7 29.18
Optimal : americas 8 29.7
Optimal : americas 9 26.53
Optimal : americas 10 21.56
Optimal : americas 11 15.88
Optimal : americas 12 11.89
Optimal : bahrain 3 20.41
Optimal : bahrain 4 24.39
Optimal : bahrain 5 28.95
Optimal : bahrain 10 29.16
Optimal : bahrain 11 24.33
Optimal : bahrain 12 19.34
Optimal : baku 4 12.13
Optimal : baku 5 17.96
Optimal : baku 6 23.38
Optimal : baku 7 26.46
Optimal : baku 8 27.0
Optimal : baku 9 22.84
Optimal : baku 10 17.71
Optimal : baku 11 12.0
Optimal : catalunya 3 11.0
Optimal : catalunya 4 13.3

Optimal : losail 5 25.8
Optimal : losail 6 28.11
Optimal : losail 9 27.63
Optimal : losail 10 24.8
Optimal : losail 11 21.12
Optimal : losail 12 16.65
Optimal : marina_bay 3 25.39
Optimal : marina_bay 4 25.72
Optimal : marina_bay 5 26.17
Optimal : marina_bay 6 26.2
Optimal : marina_bay 7 26.09
Optimal : marina_bay 8 25.92
Optimal : marina_bay 9 25.78
Optimal : marina_bay 10 25.51
Optimal : marina_bay 11 25.23
Optimal : marina_bay 12 25.03
Optimal : miami 3 19.46
Optimal : miami 4 21.33
Optimal : miami 5 23.17
Optimal : miami 6 24.9
Optimal : miami 7 25.65
Optimal : miami 8 25.81
Optimal : miami 9 25.24
Optimal : miami 10 23.79
Optimal : miami 11 21.26
Optimal : miami 12 19.33
Optimal : monaco 3 7.59
Optimal : monaco 4 10.27
Optimal : monaco 5 14.14
Optimal : monaco 6 18.08
Optimal : monaco 7 20.7
Optimal : monaco 8 20.97
Optimal : monaco 9 17.65
Optimal : monaco 10 14.29
Optimal : monaco 11 9.99
Optimal : monaco 12 6.84
Optimal : monza 4 6.56
Optimal : monza 5 11.19
Optimal : monza 6 1

Optimal : jeddah 3 10.03
Optimal : jeddah 12 10.03
Optimal : losail 3 2.07
Optimal : losail 11 11.57
Optimal : losail 12 0.0
Optimal : miami 3 1.03
Optimal : miami 4 4.37
Optimal : miami 11 3.3
Optimal : miami 12 0.3
Optimal : monaco 3 0.0
Optimal : monaco 4 0.0
Optimal : monaco 5 0.0
Optimal : monaco 6 1.13
Optimal : monaco 7 5.63
Optimal : monaco 8 5.93
Optimal : monaco 9 0.03
Optimal : monaco 10 0.0
Optimal : monaco 11 0.0
Optimal : monaco 12 0.0
Optimal : monza 3 0.0
Optimal : monza 4 0.0
Optimal : monza 5 0.07
Optimal : monza 6 2.67
Optimal : monza 7 5.97
Optimal : monza 8 5.0
Optimal : monza 9 0.0
Optimal : monza 10 0.0
Optimal : monza 11 0.0
Optimal : monza 12 0.0
Optimal : red_bull_ring 3 0.0
Optimal : red_bull_ring 4 0.0
Optimal : red_bull_ring 5 0.0
Optimal : red_bull_ring 6 0.0
Optimal : red_bull_ring 7 0.03
Optimal : red_bull_ring 8 0.1
Optimal : red_bull_ring 9 0.0
Optimal : red_bull_ring 10 0.0
Optimal : red_bull_ring 11 0.0
Optimal : red_bull_ring 12 0.0
Optimal : rodrig

snow_days
0.0 0.0 0.0
Optimal : albert_park 3 0.0
Optimal : albert_park 4 0.0
Optimal : albert_park 5 0.0
Optimal : albert_park 6 0.0
Optimal : albert_park 8 0.0
Optimal : albert_park 9 0.0
Optimal : albert_park 10 0.0
Optimal : albert_park 11 0.0
Optimal : albert_park 12 0.0
Optimal : americas 3 0.0
Optimal : americas 5 0.0
Optimal : americas 6 0.0
Optimal : americas 7 0.0
Optimal : americas 8 0.0
Optimal : americas 9 0.0
Optimal : americas 10 0.0
Optimal : bahrain 3 0.0
Optimal : bahrain 4 0.0
Optimal : bahrain 5 0.0
Optimal : bahrain 6 0.0
Optimal : bahrain 7 0.0
Optimal : bahrain 8 0.0
Optimal : bahrain 9 0.0
Optimal : bahrain 10 0.0
Optimal : bahrain 11 0.0
Optimal : bahrain 12 0.0
Optimal : baku 5 0.0
Optimal : baku 6 0.0
Optimal : baku 7 0.0
Optimal : baku 8 0.0
Optimal : baku 9 0.0
Optimal : catalunya 4 0.0
Optimal : catalunya 5 0.0
Optimal : catalunya 6 0.0
Optimal : catalunya 7 0.0
Optimal : catalunya 8 0.0
Optimal : catalunya 9 0.0
Optimal : hungaroring 5 0.0
Optimal : hunga

In [22]:
print(cols)

['temp_mean (°C)', 'temp_max (°C)', 'temp_min (°C)', 'precipitation_mean (mm)', 'rain_mean (mm)', 'snow_mean (cm)', 'precipitation_total (mm)', 'rain_total (mm)', 'snow_total (cm)', 'moderate_days (>25.55°C)', 'high_days (>27.77°C)', 'precipitation_days', 'rain_days', 'snow_days']


In [23]:
optimal_cols = ['circuitRef', 'month', 'temp_mean (°C)', 'temp_max (°C)', 'temp_min (°C)', 'precipitation_mean (mm)', 'rain_mean (mm)', 'snow_mean (cm)', 'precipitation_total (mm)', 'rain_total (mm)', 'snow_total (cm)', 'moderate_days (>25.55°C)', 'high_days (>27.77°C)', 'precipitation_days', 'rain_days', 'snow_days']

optimal_weather = pd.DataFrame(columns = optimal_cols)

for index, row in weather.iterrows():
    
    optimal_info = dict.fromkeys(optimal_cols, 0)
    
    optimal_info['circuitRef'] = row['circuitRef']
    optimal_info['month'] = row['month']
    
    for col in cols:
    
        q1 = round(race_info_weather[col].quantile(.25), 2)
        q3 = round(race_info_weather[col].quantile(.75), 2)
        iqr = round(q3-q1, 2)
        
        if row['month'] < 3:
            optimal_info[col] = 'Off-season'
            continue
        
        if row[col] <= (q3 + (1.5*iqr)):
            if row[col] >= (q1 - (1.5*iqr)):
                #print("Optimal :", row['circuitRef'], row['month'],row[col])
                optimal_info[col] = "Optimal"
                
        if row[col] > (q3 + (1.5*iqr)):
            #print("High outlier :", row['circuitRef'], row['month'],row[col])
            optimal_info[col] = "High outlier"
                       
        if row[col] < (q1 - (1.5*iqr)):
            #print("Low outlier :", row['circuitRef'], row['month'], row[col])
            optimal_info[col] = "Low outlier"
            
    
    optimal_row = pd.DataFrame([optimal_info])
        
    optimal_weather = pd.concat([optimal_weather, optimal_row], ignore_index=True)
    

In [24]:
optimal_weather

,circuitRef,month,temp_mean (°C),temp_max (°C),temp_min (°C),precipitation_mean (mm),rain_mean (mm),snow_mean (cm),precipitation_total (mm),rain_total (mm),snow_total (cm),moderate_days (>25.55°C),high_days (>27.77°C),precipitation_days,rain_days,snow_days
0,albert_park,1,Off-season,Off-season,Off-season,Off-season,Off-season,Off-season,Off-season,Off-season,Off-season,Off-season,Off-season,Off-season,Off-season,Off-season
1,albert_park,2,Off-season,Off-season,Off-season,Off-season,Off-season,Off-season,Off-season,Off-season,Off-season,Off-season,Off-season,Off-season,Off-season,Off-season
2,albert_park,3,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal
3,albert_park,4,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal
4,albert_park,5,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal
5,albert_park,6,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal
6,albert_park,7,Optimal,Optimal,Optimal,Optimal,Optimal,High outlier,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,High outlier
7,albert_park,8,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal
8,albert_park,9,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal
9,albert_park,10,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal,Optimal


In [25]:
print(optimal_weather['circuitRef'].unique().tolist())

optimal_months = {}

for circuit in optimal_weather['circuitRef'].unique().tolist():
    optimal_months[circuit] = []
    
print()
print(optimal_months)

['albert_park', 'americas', 'bahrain', 'baku', 'catalunya', 'hungaroring', 'imola', 'interlagos', 'jeddah', 'losail', 'marina_bay', 'miami', 'monaco', 'monza', 'red_bull_ring', 'rodriguez', 'shanghai', 'silverstone', 'spa', 'suzuka', 'vegas', 'villeneuve', 'yas_marina', 'zandvoort']

{'albert_park': [], 'americas': [], 'bahrain': [], 'baku': [], 'catalunya': [], 'hungaroring': [], 'imola': [], 'interlagos': [], 'jeddah': [], 'losail': [], 'marina_bay': [], 'miami': [], 'monaco': [], 'monza': [], 'red_bull_ring': [], 'rodriguez': [], 'shanghai': [], 'silverstone': [], 'spa': [], 'suzuka': [], 'vegas': [], 'villeneuve': [], 'yas_marina': [], 'zandvoort': []}


In [26]:
#print(optimal_weather['circuitRef'].unique().tolist())

optimal_months = {}

for circuit in optimal_weather['circuitRef'].unique().tolist():
    optimal_months[circuit] = []
    
suboptimal_months = {}

for circuit in optimal_weather['circuitRef'].unique().tolist():
    suboptimal_months[circuit] = []
    
#print()
#print(optimal_months)

for index, row in optimal_weather.iterrows():
    count = 0
    for col in cols:
        if row[col] == "Optimal":
            count = count + 1
    
    if count == 14:
        print(row['circuitRef'], row['month'])
        optimal_months[row['circuitRef']] = optimal_months[row['circuitRef']] + [row['month']]
    
    """
    elif count == 13:
        if optimal_weather.loc[index, 'moderate_days (>25.55°C)'] != "Optimal":
            print(row['circuitRef'], row['month'])
            optimal_months[row['circuitRef']] = optimal_months[row['circuitRef']] + [row['month']]
    """
    
    else:
        suboptimal_months[row['circuitRef']] = suboptimal_months[row['circuitRef']] + [[row['month'], count]]

SyntaxError: invalid syntax (1334587971.py, line 33)

In [ ]:
print(len(cols))

In [ ]:
optimal_months

In [ ]:
optimal_months

In [ ]:
for window in suboptimal_months['marina_bay']:
    print(window[0], ":", window[1])

In [ ]:
for window in suboptimal_months['suzuka']:
    print(window[0], ":", window[1])

In [ ]:
for i in range(3, 13):
    print(i)
    for circuit in optimal_months.keys():
        if i in optimal_months[circuit]:
            print(circuit)
            
    print()